In [1]:
import chseg
import numpy as np
import tensorflow as tf

from sklearn.metrics import classification_report

In [2]:
params = {
    'seq_len': 50,
    'batch_size': 128,
    'n_class': 4,
    'kernel_sizes': [3, 5],
    'hidden_dim': 128,
    'text_iter_step': 10,
    'lr': {'start': 5e-3, 'end': 5e-4}
}

In [3]:
def to_test_seq(*args):
    return [np.reshape(x[:(len(x)-len(x)%params['seq_len'])],
        [-1,params['seq_len']]) for x in args]

def iter_seq(x):
    return np.array([x[i: i+params['seq_len']] for i in range(
        0, len(x)-params['seq_len'], params['text_iter_step'])])

def to_train_seq(*args):
    return [iter_seq(x) for x in args]

In [4]:
def forward(x, reuse, is_training):
    with tf.variable_scope('model', reuse=reuse):
        x = tf.contrib.layers.embed_sequence(x, params['vocab_size'], params['hidden_dim'])
        x = tf.layers.dropout(x, 0.1, training=is_training)
        
        pad = tf.zeros([tf.shape(x)[0], 1, params['hidden_dim']])
        for k_sz in params['kernel_sizes']:
            n = (k_sz - 1) // 2
            _x = tf.concat([pad]*n + [x] + [pad]*n, 1)
            x += tf.layers.conv1d(_x, params['hidden_dim'], k_sz, activation=tf.nn.relu)

        logits = tf.layers.dense(x, params['n_class'])
    return logits


def model_fn(features, labels, mode, params):
    logits_tr = forward(features, reuse=False, is_training=True)
    logits_te = forward(features, reuse=True, is_training=False)
    seq_lens = tf.count_nonzero(features, 1)
    
    if mode == tf.estimator.ModeKeys.TRAIN:
        log_likelihood, _ = tf.contrib.crf.crf_log_likelihood(
            logits_tr, labels, seq_lens)
        
        loss_op = tf.reduce_mean(-log_likelihood)
        
        global_step=tf.train.get_global_step()
        
        lr_op = tf.train.exponential_decay(
            params['lr']['start'], global_step, params['lr']['steps'],
            params['lr']['end']/params['lr']['start'])
        
        train_op = tf.train.AdamOptimizer(lr_op).minimize(
            loss_op, global_step=global_step)
        
        lth = tf.train.LoggingTensorHook({'lr': lr_op}, every_n_iter=100)
        
        return tf.estimator.EstimatorSpec(
            mode=mode, loss=loss_op, train_op=train_op, training_hooks=[lth])
    
    if mode == tf.estimator.ModeKeys.PREDICT:
        trans_params = tf.get_variable('transitions', [params['n_class'], params['n_class']])
        
        viterbi_seq, _ = tf.contrib.crf.crf_decode(
            logits_te, trans_params, seq_lens)
        
        return tf.estimator.EstimatorSpec(mode, predictions=viterbi_seq)

In [5]:
x_train, y_train, x_test, y_test, params['vocab_size'], word2idx, idx2word = chseg.load_data()
X_train, Y_train = to_train_seq(x_train, y_train)
X_test, Y_test = to_test_seq(x_test, y_test)
params['lr']['steps'] = len(X_train) // params['batch_size']

estimator = tf.estimator.Estimator(model_fn, params=params)

estimator.train(tf.estimator.inputs.numpy_input_fn(
    X_train, Y_train,
    batch_size = params['batch_size'],
    shuffle = True))

Y_pred = np.concatenate(list(estimator.predict(tf.estimator.inputs.numpy_input_fn(
    X_test,
    batch_size = params['batch_size'],
    shuffle = False))), 0)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/var/folders/sx/fv0r97j96fz8njp14dt5g7940000gn/T/tmp4nh9f0gu', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x11ad413c8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
Instructions for updating:
`NHWC` for data_format is deprecated, use `NWC` instead


/usr/local/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:98: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into /var/folders/sx/fv0r97j96fz8njp14dt5g7940000gn/T/tmp4nh9f0gu/model.ckpt.
INFO:tensorflow:loss = 75.188736, step = 1
INFO:tensorflow:lr = 0.005
INFO:tensorflow:global_step/sec: 9.59397
INFO:tensorflow:loss = 17.002552, step = 101 (10.425 sec)
INFO:tensorflow:lr = 0.004086997 (10.424 sec)
INFO:tensorflow:global_step/sec: 9.75695
INFO:tensorflow:loss = 8.168403, step = 201 (10.249 sec)
INFO:tensorflow:lr = 0.0033407088 (10.249 sec)
INFO:tensorflow:global_step/sec: 9.10521
INFO:tensorflow:loss = 7.9680376, step = 301 (10.982 sec)
INFO:tensorflow:lr = 0.0027306937 (10.982 sec)
INFO:tensorflow:global_step/sec: 9.41664
INFO:tensorflow:loss = 8.756846, step = 401 (10.619 sec)
INFO:tensorflow:lr = 0.0022320673 (10.620 sec)
INFO:tensorflow:global_step

In [6]:
print(classification_report(Y_test.ravel(), Y_pred.ravel(), target_names=['B','M','E','S']))

sample = '我来到大学读书，希望学到知识'
labels = list(estimator.predict(tf.estimator.inputs.numpy_input_fn(
    np.atleast_2d([word2idx[w] for w in sample] + [0]*(params['seq_len']-len(sample))),
    shuffle = False)))[0]

labels = labels[:len(sample)]
res = ''
for i, l in enumerate(labels):
    c = sample[i]
    if l == 2 or l == 3:
        c += ' '
    res += c
print(res)

             precision    recall  f1-score   support

          B       0.93      0.95      0.94    116058
          M       0.87      0.78      0.82     25425
          E       0.93      0.96      0.94    116057
          S       0.94      0.92      0.93    106810

avg / total       0.93      0.93      0.93    364350

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /var/folders/sx/fv0r97j96fz8njp14dt5g7940000gn/T/tmp4nh9f0gu/model.ckpt-1143
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
我 来到 大学 读书 ， 希望 学 到 知识
